**Configuração e Autenticação**

In [ ]:
import os

from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery

import pandas as pd

import requests
import base64

**Conexão Big Query**

In [ ]:
project_id = 'case-gb-elcio'
dataset_id = 'spotify'

In [ ]:
client = bigquery.Client(project=project_id)

In [ ]:
def upload_to_bq(df, table_name):
    job_config = bigquery.LoadJobConfig(
        autodetect=True
    )

    job = client.load_table_from_dataframe(
        df, f'{project_id}.{dataset_id}.{table_name}', job_config=job_config
    )
    job.result()

    print(f"Tabela {table_name} carregada com sucesso!")

**Conectando a API do Spotify e buscando token**

In [ ]:
client_id = os.environ['client_id']
client_secret = os.environ['client_secret']

client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode()).decode()

auth_url = "https://accounts.spotify.com/api/token"

auth_headers = {
    "Authorization": f"Basic {client_creds_b64}",
    "Content-Type": "application/x-www-form-urlencoded"
}

auth_data = {
    "grant_type": "client_credentials"
}

auth_response = requests.post(auth_url, headers=auth_headers, data=auth_data)

if auth_response.status_code == 200:
    access_token = auth_response.json()["access_token"]
else:
    print("Erro na autenticação")

**Função que pesquisa pelo podcast**

In [ ]:
def pesquisar_spotify(termo, tipo, market="BR", offset=0):

    search_url = f"https://api.spotify.com/v1/search?q={termo}&type={tipo}&market={market}&limit=50&offset={offset}"

    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.get(search_url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro na pesquisa: {response.status_code}")
        return None

**Função que trás os 50 primeiros resultados não nulos**

In [ ]:
def search_podcasts(termo):
    podcasts = []
    offset = 0
    while len(podcasts) < 50:
        resultados = pesquisar_spotify(termo, "show", market="BR", offset=offset)
        if resultados:
            shows = resultados.get("shows", {}).get("items", [])
            for show in shows:
                if show:
                    name = show.get("name", "")
                    description = show.get("description", "")
                    show_id = show.get("id", "")
                    total_episodes = show.get("total_episodes", "")
                    podcasts.append({"name": name, "description": description, "id": show_id, "total_episodes": total_episodes})
        else:
            break

        offset += 50

    return podcasts[:50]

**Criando o pdDF e salvando a tabela no BigQuery**

In [ ]:
podcasts_data_hackers = search_podcasts("data hackers")

df_podcasts_data_hackers = pd.DataFrame(podcasts_data_hackers)

print(df_podcasts_data_hackers.head())

In [ ]:
upload_to_bq(df_podcasts_data_hackers, 'tb_podcasts_data_hackers')